# Running Computational Alchemy for Catalysis: Transition State Barrier Height Predictions
This notebook outlines Python functions that use information from VASP calculations to predict descriptors for hypothetical catalyst. These functions utilize tools from the Atomic Simulation Environment (ASE) to manipulate catalyst surface models and perform calculations with computational alchemy. 

In this example, we apply computational alchemy on reference set of nudged elastic band (NEB) calculcations of CH4 dehydrogenation on Pt(111) to predict transition state barrier heights of the same process on hypothetical alloys of Pt(111). 

In [1]:
import comp_alchemy

In [2]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.annotations import Slope
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook

This example uses an NEB calculation with 10 images in total. Below we plot the total energy for each image. You also can hover your mouse over any data point to visualize the image

In [3]:
images = range(10)
image_energies = pd.read_csv('reference_images_energies.csv')
image_energies = image_energies['energy']

import csv

with open('reference_images_energies.csv') as refens:
    refen_read = csv.reader(refens, delimiter=',')
    it_ref = iter(refen_read)
    next(it_ref)
    image_energies = [float(row[1]) for row in it_ref]

In [4]:
output_notebook()

en_source = ColumnDataSource(
data=dict(
x=[int(im) for im in images],
y=image_energies,
ads_imgs=['neb_images/'+str(im)+'/'+'neb_image_'+str(im)+'.png' for im in images]))

en_tooltips="""
            <p>
                <span style="font-size: 25px;">NEB Image </span>
                <span style="font-size: 25px;">$index</span>
            </p>
            <p>
                <img
                src="@ads_imgs" height="150" alt="@ads_imgs" width="150"
                style="float: center; margin: 0px 15px 15px 0px;"
                border="2"
                ></img>
"""

ef = figure(plot_width=900, plot_height=600,y_range=(min(image_energies)-0.5,max(image_energies)+0.5),tooltips=en_tooltips)

ef.xaxis.axis_label = "NEB Image Number"
ef.xaxis.axis_label_text_font_style = "bold"
ef.xaxis.axis_label_text_font_size = "12pt"
ef.yaxis.axis_label = "Energy (eV)"
ef.yaxis.axis_label_text_font_style = "bold"
ef.yaxis.axis_label_text_font_size = "12pt"
ef.title.text_font_size = '12pt'
ef.title.align = 'center'

ef.circle('x', 'y', size=45, source=en_source)
ef.line('x','y', line_dash="dashed", source=en_source)

show(ef, notebook_handle=True)

Loading BokehJS ...

From the plot above, it's apparent that image 5 is the transition state for CH4 dehydrogenation.

Next we make ASE Atoms objects with VASP POSCARs and CONTCARs for the bare catalyst surface (slab) and the catalyst surface with the adsorbate (ads) for each image. These models and the results of the VASP calculations for each will be used as our reference for computational alchemy calculations.

In [5]:
from ase.io import read 
from ase.visualize import view
slab = read('slab/POSCAR',format='vasp')
slab_contcar = read('slab/CONTCAR',format='vasp')
ads = [read(f'neb_images/{str(im)}/POSCAR',format='vasp') for im in images]
ads_contcar = [read(f'neb_images/{str(im)}/CONTCAR',format='vasp') for im in images]

In [6]:
from comp_alchemy.ads_slab_pairs import pairs
p = [pairs(slab,slab) for a in ads]

### Working with electrostatic potentials in VASP OUTCAR
Next we can grab the atom-centered electrostatic potentials (ESPs) printed at the end of the OUTCAR with `grab_esp()`. 
Here we grab ESPs for slab and ads and calculate the differences with `espdiff()`. The ESP differnces define the alchemical derivatives that facilitate predictions with alchemy.

In [7]:
from comp_alchemy.elec_stat_pot import (grab_esp,espdiff,remove_duplicate_espdiffs,heatmap)
slab_elec = grab_esp(poscar='slab/POSCAR',outcar='slab/trunc-OUTCAR')
ads_elec = [grab_esp(poscar=f'neb_images/{str(im)}/POSCAR',
                     outcar=f'neb_images/{str(im)}/trunc-OUTCAR') for im in images]
diffs = [espdiff(elec1=slab_elec,elec2=ads_elec[im],pair=p[im]) for im in images]

In [8]:
diffs0 = [round(d,4) for d in diffs[0]]
print(f'''
Atom-centered electrostatic potentials for the initial state of CH4 dehydrogenation (image 0):

Slab:
{slab_elec}

Ads:
{ads_elec[0]}

Differences in electrostatic potentials between ads and slab
{diffs0}''')


Atom-centered electrostatic potentials for the initial state of CH4 dehydrogenation (image 0):

Slab:
[-73.7064, -73.7064, -73.7064, -73.7064, -74.1493, -74.1493, -74.1493, -74.1493, -74.0957, -74.0957, -74.0957, -74.0957, -73.6192, -73.6192, -73.6192, -73.6192]

Ads:
[-74.1863, -74.186, -74.1864, -74.1866, -74.6565, -74.6556, -74.6567, -74.6575, -74.6527, -74.6519, -74.6519, -74.6528, -74.1955, -74.1747, -74.1757, -74.1765, -53.2355, -36.6325, -36.6332, -36.6324, -36.6331]

Differences in electrostatic potentials between ads and slab
[-0.4799, -0.4796, -0.48, -0.4802, -0.5072, -0.5063, -0.5074, -0.5082, -0.557, -0.5562, -0.5562, -0.5571, -0.5763, -0.5555, -0.5565, -0.5573]


### Visualizing alchemical derivatives
With the `heatmap()` function, visualize the alchemical derivatives with ASE GUI by running the cell below, pressing "c" in ASE GUI, and under "Choose how the atoms are colored:" selecting "By initial charge" 

In [9]:
slab_vis = [heatmap(poscar='slab/POSCAR',dexlist=[dex[0] for dex in p[im]],espdiffs=diffs[im]) for im in range(0,len(images))]
#view(slab_vis)

### Creating hypothetical materials by making transmutations
Below we use the `index_transmuted()` function to grab indexes of atoms that we want to transmute in our reference slabs. We specify the number of transmutable atoms by setting `transmute_num`. 

This function also grabs indexes of atoms at the bottom of the slab that we can counter-transmute to maintain isoelectronicity. We set the number of counter-transmutable atoms with `counter_num`.

In [10]:
from ase import Atom
from comp_alchemy.alloy_index import index_transmuted, transmuter, transmuted_directory_names
metal = Atom('Pt')
[transmute, counter] = index_transmuted(slab=slab,
                                        transmute_atom_sym=metal.symbol,
                                        counter_atom_sym=metal.symbol,
                                        transmute_num=8,
                                        counter_num=1,
                                        symmetric = False)

new_transmute = [transmute for im in images]
new_counter = [counter for im in images]

In [11]:
print(f'''Indices of atoms to be transmuted in the top two surface layers.
{transmute}

Indices of atoms to be counter transmuted with unique electrostatic potential differences.
{counter}

There are {len(transmute)} sites near the surface of the catalyst where we will make transmutations.

For each transmutation, we can make counter transmutations at 1 of {len(counter)} sites.

Considering all combinations of transmute and counter-transmute sites, we will assess a total of {len(transmute)*len(counter)} hypothetical configurations of alloys.
''')

Indices of atoms to be transmuted in the top two surface layers.
[12, 13, 14, 15, 8, 9, 10, 11]

Indices of atoms to be counter transmuted with unique electrostatic potential differences.
[0]

There are 8 sites near the surface of the catalyst where we will make transmutations.

For each transmutation, we can make counter transmutations at 1 of 1 sites.

Considering all combinations of transmute and counter-transmute sites, we will assess a total of 8 hypothetical configurations of alloys.



In this example, we transmute surface Pt atoms to Au (change in nuclear charge of +1) and counter-transmute Pt atoms in the bottom layers to Ir (change in nuclear charge of -1). Below we construct an array with Au and Ir Atom objects to use in the `transmuter()` function.

In [12]:
charge = 1
transmute_atom = Atom(metal.symbol)
transmute_atom.number += charge
counter_transmute_atom = Atom(metal.symbol)
counter_transmute_atom.number -= charge
all_transmute = [counter_transmute_atom,transmute_atom]

Next, we make transmutations to Atoms objects of slab and ads from their respective CONTCARs and give labels for each new model with the `transmuted_directory_names()` function. Now we can visualize these new systems and write POSCARs to do VASP calculations for benchmarking.

In [13]:
transmuted_slabs = []
dir_slab = []
num = 0
for im in range(0,len(images)):
    transmuted_single_image = []
    dir_single_image = []
    for i,c in enumerate(new_counter[im]):
        for j,t in enumerate(new_transmute[im]):
            transmuted_single_image.append(transmuter(slab=slab_contcar,
                                                     atomdex=[c,t],
                                                     trans=all_transmute))
            dir_single_image.append(transmuted_directory_names(bdex=i,
                                                              tdex=j,
                                                              dexes=[c,t],
                                                              atoms_array=all_transmute))
    transmuted_slabs.append(transmuted_single_image)
    dir_slab.append(dir_single_image)

In [14]:
transmuted_ads = []
dir_ads = []
num = 0
for im in range(0,len(images)):
    transmuted_single_image = []
    dir_single_image = []
    for i,c in enumerate(new_counter[im]):
        for j,t in enumerate(new_transmute[im]):
            transmuted_single_image.append(transmuter(slab=ads_contcar[im],
                                                     atomdex=[c,t],
                                                     trans=all_transmute))
            dir_single_image.append(transmuted_directory_names(bdex=i,
                                                              tdex=j,
                                                              dexes=[c,t],
                                                              atoms_array=all_transmute))
    transmuted_ads.append(transmuted_single_image)
    dir_ads.append(dir_single_image)

# After VASP benchmarking calculations

In [15]:
from comp_alchemy.read_oszicar import grab_energy
adsorbate_energy = 0
slab_energy = grab_energy(oszicar='slab/trunc-OSZICAR')
ads_energy = image_energies
ref_be = [slab_energy + adsorbate_energy - ae for ae in ads_energy]

In [16]:
import csv

with open('transmuted_slab_energies.csv') as slaben:
    slaben_reader = csv.reader(slaben, delimiter=',')
    it_s = iter(slaben_reader)
    next(it_s)
    transmuted_slab_energy = []
    image_energies = []
    ener_num = np.min([len(d) for d in dir_slab])
    for row in it_s:
        image_energies.append(float(row[2]))
        ener_num -= 1
        if ener_num == 0:
            transmuted_slab_energy.append(image_energies)
            ener_num = np.min([len(d) for d in dir_slab])
            image_energies =  []

In [17]:
import csv

with open('transmuted_ads_energies.csv') as adsen:
    adsen_reader = csv.reader(adsen, delimiter=',')
    it_a = iter(adsen_reader)
    next(it_a)
    transmuted_ads_energy = []
    image_energies = []
    ener_num = np.min([len(d) for d in dir_ads])
    for row in it_a:
        image_energies.append(float(row[2]))
        ener_num -= 1
        if ener_num == 0:
            transmuted_ads_energy.append(image_energies)
            ener_num = np.min([len(d) for d in dir_ads])
            image_energies =  []

In [18]:
transmuted_be = [[transmuted_slab_energy[im][i] + adsorbate_energy - transmuted_ads_energy[im][i] for i in range(len(transmuted_slab_energy[im]))] for im in range(len(images))]
dft_del_be = [[round(transmuted_be[im][i] - ref_be[im],4) for i in range(len(transmuted_be[im]))] for im in range(len(images))]

In [19]:
from comp_alchemy.binding_energy import alc_be
dn = []
alc_del_be = []
for im in range(len(images)):
    dn_single_image = []
    alc_del_be_single_image = []
    for j in range(len(new_counter[im])):
        dn_single_image.append([alc_be(transmute=[new_transmute[im][i]],
             counter=[new_counter[im][j]],
             espdiffs=diffs[im],
             charge=charge)[0] for i in range(len(new_transmute[im]))])
        
        alc_del_be_single_image.append([alc_be(transmute=[new_transmute[im][i]],
                     counter=[new_counter[im][j]],
                     espdiffs=diffs[im],
                     charge=charge)[1] for i in range(len(new_transmute[im]))])
    
    dn_sum = []
    for dn_i in dn_single_image:
        dn_sum += dn_i
    alc_sum = []
    for alc_i in alc_del_be_single_image:
        alc_sum += alc_i
        
    dn.append(dn_sum)
    alc_del_be.append(alc_sum)

In [20]:
new_transmute_espdiffs = [[diffs[im][i] for i in new_transmute[im]] for im in range(len(images))]
new_counter_espdiffs = [[diffs[im][i] for i in new_counter[im]] for im in range(len(images))]

In [21]:
from numpy import average
ae = [[round(abs(alc_del_be[im][i] - dft_del_be[im][i]),3) for i in range(len(alc_del_be[im]))] for im in range(len(images))]
mae = [round(average(ae[im]),3) for im in range(len(images))]

In [22]:
output_notebook()

i = 0

be_source = ColumnDataSource(
data=dict(
x=dft_del_be[i],
y=alc_del_be[i],
desc=dir_slab[i],
slab_imgs=['slab/'+d+'.png' for d in dir_slab[i]],
ads_imgs=['neb_images/'+str(i)+'/'+a+'.png' for a in dir_slab[i]],
aerr=ae[i]))

be_tooltips="""
            <p>
                <span style="font-size: 25px;">Surface Index:</span>
                <span style="font-size: 25px;">$index</span>
            </p>
            <p>
                <img
                src="@slab_imgs" height="100" alt="@slab_imgs" width="100"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
                ></img>
                <img
                src="@ads_imgs" height="150" alt="@ads_imgs" width="150"
                style="float: center; margin: 0px 15px 15px 0px;"
                border="2"
                ></img>
            <p>
                <span style="font-size: 25px;">Transmute Label:</span>
                <span style="font-size: 25px; font-weight: bold;">@desc</span>
            </p>
            <p>
                <span style="font-size: 25px;">DFT, Alchemy:</span>
                <span style="font-size: 25px; font-weight: bold;">@x eV, @y eV</span>
            </p>
            <p>
                <span style="font-size: 25px;">Absolute Error:</span>
                <span style="font-size: 25px; font-weight: bold;">@aerr eV</span>
            </p>
"""

pf = figure(plot_width=800, plot_height=800,x_range=(min(dft_del_be[i]+alc_del_be[i])-0.5,max(dft_del_be[i]+alc_del_be[i])+0.5),
           y_range=(min(dft_del_be[i]+alc_del_be[i])-0.5,max(dft_del_be[i]+alc_del_be[i])+0.5),
            tooltips=be_tooltips,
            title="Parity Plot for BE Predictions on NEB Images with Computational Alchemy")

pf.xaxis.axis_label = "DFT"
pf.xaxis.axis_label_text_font_style = "bold"
pf.xaxis.axis_label_text_font_size = "12pt"
pf.yaxis.axis_label = "Alchemy"
pf.yaxis.axis_label_text_font_style = "bold"
pf.yaxis.axis_label_text_font_size = "12pt"
pf.title.text_font_size = '12pt'
pf.title.align = 'center'

r = pf.circle('x', 'y', size=45, source=be_source)

parity = Slope(gradient=1,y_intercept=0,)
pf.add_layout(parity)

Loading BokehJS ...

In [23]:
def update(neb_image):
    neb_image = int(neb_image)
    r.data_source.data['x'] = dft_del_be[neb_image]
    r.data_source.data['y'] = alc_del_be[neb_image]
    push_notebook()

In [24]:
show(pf, notebook_handle=True)

In [31]:
interact(update, neb_image=[int(n) for n in np.linspace(0,9,10)])

interactive(children=(Dropdown(description='neb_image', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), Outp…

<function __main__.update(neb_image)>

In [26]:
for i in range(len(dft_del_be[0])):
    print(dft_del_be[0][i],alc_del_be[0][i],ae[0][i])

-0.0475 -0.0963999999999885 0.049
-0.0646 -0.07559999999999434 0.011
-0.0501 -0.07659999999999911 0.026
-0.0524 -0.07739999999999725 0.025
-0.0473 -0.0771000000000015 0.03
-0.0463 -0.07630000000000337 0.03
-0.0463 -0.07630000000000337 0.03
-0.0466 -0.07720000000000482 0.031


In [27]:
print(mae)

[0.029, 0.031, 0.035, 0.053, 0.092, 0.22, 0.255, 0.264, 0.267, 0.267]


In [28]:
dft_del_ea = [dft_del_be[5][i]-dft_del_be[0][i] for i in range(len(dft_del_be[0]))]
alc_del_ea = [alc_del_be[5][i]-alc_del_be[0][i] for i in range(len(alc_del_be[0]))]

In [29]:
output_notebook()

i = 0

ae_source = ColumnDataSource(
data=dict(
x=dft_del_ea,
y=alc_del_ea,
desc=dir_slab[i]))

ae_tooltips="""
            <p>
                <span style="font-size: 25px;">Surface Index:</span>
                <span style="font-size: 25px;">$index</span>
            </p>
            <p>
                <span style="font-size: 25px;">Transmute Label:</span>
                <span style="font-size: 25px; font-weight: bold;">@desc</span>
            </p>
            <p>
                <span style="font-size: 25px;">DFT, Alchemy:</span>
                <span style="font-size: 25px; font-weight: bold;">@x eV, @y eV</span>
            </p>
"""

af = figure(plot_width=800, plot_height=800,x_range=(min(dft_del_ea+alc_del_ea)-0.5,max(dft_del_ea+alc_del_ea)+0.5),
           y_range=(min(dft_del_ea+alc_del_ea)-0.5,max(dft_del_ea+alc_del_ea)+0.5),
            tooltips=ae_tooltips,
            title="Parity Plot for Barrier Height Predictions with Computational Alchemy")

af.xaxis.axis_label = "DFT"
af.xaxis.axis_label_text_font_style = "bold"
af.xaxis.axis_label_text_font_size = "12pt"
af.yaxis.axis_label = "Alchemy"
af.yaxis.axis_label_text_font_style = "bold"
af.yaxis.axis_label_text_font_size = "12pt"
af.title.text_font_size = '12pt'
af.title.align = 'center'

af.circle('x', 'y', size=45, source=ae_source)

parity = Slope(gradient=1,y_intercept=0,)
af.add_layout(parity)
show(af, notebook_handle=True)

Loading BokehJS ...

In [30]:
print(dft_del_ea)
print(alc_del_ea)

[-0.014100000000000001, -0.8229, 0.003599999999999999, -0.05349999999999999, -0.0381, -0.0743, 0.0015000000000000013, 0.017300000000000003]
[-0.7267000000000081, -0.05310000000000059, -0.011499999999998067, -0.033699999999996066, 0.03649999999998954, 0.011199999999988108, -0.00940000000001362, -0.006100000000003547]
